In [1]:
#全局变量
hub_token = open('/root/hub_token.txt').read().strip()
repo_id = 'lansinuote/nlp.1.predict_last_word'
push_to_hub = True

In [2]:
from transformers import AutoTokenizer

#加载编码器
tokenizer = AutoTokenizer.from_pretrained('distilgpt2', use_fast=True)

print(tokenizer)

#编码试算
tokenizer.batch_encode_plus([
    'hide new secretions from the parental units',
    'contains no wit , only labored gags'
])

GPT2TokenizerFast(name_or_path='distilgpt2', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'})


{'input_ids': [[24717, 649, 3200, 507, 422, 262, 21694, 4991], [3642, 1299, 645, 20868, 837, 691, 2248, 1850, 308, 3775]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [3]:
from datasets import load_dataset


def get_dataset():
    #加载数据
    dataset = load_dataset(path='glue', name='sst2')

    #分词,同时删除多余的字段
    def f(data):
        return tokenizer.batch_encode_plus(data['sentence'])

    dataset = dataset.map(f,
                          batched=True,
                          batch_size=1000,
                          num_proc=4,
                          remove_columns=['sentence', 'idx', 'label'])

    #过滤掉太短的句子
    def f(data):
        return [len(i) >= 8 for i in data['input_ids']]

    dataset = dataset.filter(f, batched=True, batch_size=1000, num_proc=4)

    #截断句子,同时整理成模型需要的格式
    def f(data):
        data['input_ids'] = [i[:8] for i in data['input_ids']]
        data['attention_mask'] = [[1] * 8] * len(data['attention_mask'])
        #在模型中处理了偏移量问题,这里保持输入输出一致即可
        data['labels'] = data['input_ids'].copy()
        return data

    dataset = dataset.map(f, batched=True, batch_size=1000, num_proc=4)

    return dataset


if push_to_hub:
    dataset = get_dataset()
    dataset.push_to_hub(repo_id=repo_id, token=hub_token)

#直接使用我处理好的数据集
dataset = load_dataset(path=repo_id)

dataset, dataset['train'][0]

Found cached dataset glue (/root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-dfcf13f88e3dfa37.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-22588aea6bfeca67.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-77fd75f5f6f72f1c.arrow


...

Generating test split:   0%|          | 0/1730 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/39905 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/lansinuote___parquet/lansinuote--nlp.1.predict_last_word-4649910dfc3a98e4/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

(DatasetDict({
     validation: Dataset({
         features: ['input_ids', 'attention_mask', 'labels'],
         num_rows: 848
     })
     test: Dataset({
         features: ['input_ids', 'attention_mask', 'labels'],
         num_rows: 1730
     })
     train: Dataset({
         features: ['input_ids', 'attention_mask', 'labels'],
         num_rows: 39905
     })
 }),
 {'input_ids': [24717, 649, 3200, 507, 422, 262, 21694, 4991],
  'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1],
  'labels': [24717, 649, 3200, 507, 422, 262, 21694, 4991]})

In [4]:
import torch
from transformers.data.data_collator import default_data_collator

#数据加载器
loader = torch.utils.data.DataLoader(
    dataset=dataset['train'],
    batch_size=8,
    collate_fn=default_data_collator,
    shuffle=True,
    drop_last=True,
)

for i, data in enumerate(loader):
    break

len(loader), data

(4988,
 {'input_ids': tensor([[  259,   257,  2565,   837,   326,   705,    82,   257],
          [ 1169,  2646,   705,    82,  6218,   286, 15621,   290],
          [   64,  1598,    12, 18834, 18560,   286,   281, 31146],
          [   64,  3105,    12, 31462,  1644,    12,  1676,   771],
          [  272,   512,   273,  1346, 38411,   605, 10997,   326],
          [   86,   271,  8463,   318,  5827,   290, 34318, 47112],
          [  271,   530,   286,   995, 22041,   705,    82,   749],
          [   72,  2497,   340,   355,   257,  1862,  2933,   220]]),
  'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1, 1]]),
  'labels': tensor([[  259,   257,  2565,   837,   326,   705,    82,   257],
          [ 1169,  2646,   705,    82,  6218,   

In [5]:
from transformers import AutoModelForCausalLM, GPT2Model, PreTrainedModel, PretrainedConfig

#加载模型
#model = AutoModelForCausalLM.from_pretrained('distilgpt2')


#定义下游任务模型
class Model(PreTrainedModel):
    config_class = PretrainedConfig

    def __init__(self, config):
        super().__init__(config)
        self.pretrained = GPT2Model.from_pretrained('distilgpt2')
        self.fc = torch.nn.Linear(768, tokenizer.vocab_size, bias=False)

        #加载预训练模型的参数
        parameters = AutoModelForCausalLM.from_pretrained('distilgpt2')
        self.fc.load_state_dict(parameters.lm_head.state_dict())

        self.criterion = torch.nn.CrossEntropyLoss()

    def forward(self, input_ids, attention_mask, labels=None):
        logits = self.pretrained(input_ids=input_ids,
                                 attention_mask=attention_mask)
        logits = logits.last_hidden_state

        logits = self.fc(logits)

        loss = None
        if labels is not None:
            shift_logits = logits[:, :-1].reshape(-1, tokenizer.vocab_size)
            shift_labels = labels[:, 1:].reshape(-1)

            loss = self.criterion(shift_logits, shift_labels)

        return {'loss': loss, 'logits': logits}


model = Model(PretrainedConfig())

#统计参数量
print(sum(i.numel() for i in model.parameters()) / 10000)

out = model(**data)

out['loss'], out['logits'].shape

Some weights of the model checkpoint at distilgpt2 were not used when initializing GPT2Model: ['lm_head.weight']
- This IS expected if you are initializing GPT2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


12050.9952


(tensor(5.6376, grad_fn=<NllLossBackward0>), torch.Size([8, 8, 50257]))

In [6]:
#测试
def test():
    model.eval()

    #数据加载器
    loader_test = torch.utils.data.DataLoader(
        dataset=dataset['test'],
        batch_size=8,
        collate_fn=default_data_collator,
        shuffle=True,
        drop_last=True,
    )

    correct = 0
    total = 0
    for i, data in enumerate(loader_test):
        #只计算最后一个词的正确率,这里先把最后一个词取出来
        label = data['input_ids'][:, -1].clone()

        #从数据中抹除掉最后一个词,防止模型作弊
        data['input_ids'][:, -1] = 0

        #label就不需要了
        data['labels'][:, :] = 0

        #计算
        with torch.no_grad():
            out = model(**data)

        #只计算最后一个词的正确率,因为有偏移量的关系,这里取的是倒数第二个词
        out = out['logits'].argmax(dim=2)[:, -2]

        correct += (label == out).sum().item()
        total += 8

        if i % 10 == 0:
            print(i)
            print(label)
            print(out)

        if i == 50:
            break

    print(correct / total)

    for i in range(8):
        print(tokenizer.decode(data['input_ids'][i, :-1]))
        print(tokenizer.decode(label[i]), tokenizer.decode(out[i]))
        print()


test()

0
tensor([7297, 5667,  837,  303,  306,  705, 4661,  307])
tensor([3807,  318,   13,  303,  290,   13, 4007,  307])
10
tensor([  389,   481,  4756,   991,   837, 10224, 24870,    82])
tensor([ 547,   13, 1613,  314,   11,  290,  262,  464])
20
tensor([  286,   284,   318,   306,   257,   717,   262, 15444])
tensor([286, 466, 318, 837, 257, 717, 257, 880])
30
tensor([27954,   837,   287,   345,   290,   345,   837,   764])
tensor([262,  13, 287, 262, 290, 345, 290, 287])
40
tensor([16316, 42262,    12,  2984,   837,  2491,   764,   966])
tensor([ 318, 2274, 5069,  351,   11, 2646,   13,  966])
50
tensor([4600, 3762,  837, 2700,  557,  262,  290, 2915])
tensor([  198, 38393,   290,  8737, 32745,   262,    11,    12])
0.19117647058823528
dreary, highly annoying...
 ` 


an involving true story of a ch
inese ubby

no number of fantastic sets, extras
,  and

a work of astonishing delicacy and
 force  beauty

credibility sinks into a m
ire uddled

... ( like ) channel surfing between
 the  t

In [7]:
from transformers import AdamW
from transformers.optimization import get_scheduler


#训练
def train():
    optimizer = AdamW(model.parameters(), lr=2e-5)
    scheduler = get_scheduler(name='linear',
                              num_warmup_steps=0,
                              num_training_steps=len(loader),
                              optimizer=optimizer)

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.train()
    model.to(device)
    for i, data in enumerate(loader):
        for k in data.keys():
            data[k] = data[k].to(device)

        out = model(**data)
        loss = out['loss']

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

        optimizer.zero_grad()
        model.zero_grad()

        if i % 50 == 0:
            labels = data['labels'][:, 1:]
            out = out['logits'].argmax(dim=2)[:, :-1]

            correct = (labels == out).sum().item()
            accuracy = correct / (8 * 7)

            lr = optimizer.state_dict()['param_groups'][0]['lr']

            print(i, loss.item(), accuracy, lr)

    model.to('cpu')


if push_to_hub:
    train()
    model.push_to_hub(repo_id=repo_id, use_auth_token=hub_token)

/root/anaconda3/envs/pt39/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


0 6.208362102508545 0.16071428571428573 1.999599037690457e-05
50 5.238093376159668 0.21428571428571427 1.979550922213312e-05
100 5.600107669830322 0.19642857142857142 1.959502806736167e-05
150 5.695983409881592 0.17857142857142858 1.939454691259022e-05
200 5.34975528717041 0.17857142857142858 1.9194065757818766e-05
250 5.648352146148682 0.17857142857142858 1.8993584603047316e-05
300 5.717629909515381 0.14285714285714285 1.8793103448275863e-05
350 5.948981761932373 0.14285714285714285 1.8592622293504413e-05
400 5.349982738494873 0.16071428571428573 1.839214113873296e-05
450 5.168151378631592 0.21428571428571427 1.819165998396151e-05
500 4.825538158416748 0.26785714285714285 1.7991178829190057e-05
550 4.234778881072998 0.23214285714285715 1.7790697674418608e-05
600 4.56860876083374 0.23214285714285715 1.7590216519647154e-05
650 4.447201728820801 0.25 1.7389735364875705e-05
700 4.872074604034424 0.25 1.718925421010425e-05
750 4.521966457366943 0.32142857142857145 1.6988773055332802e-05
80

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/488M [00:00<?, ?B/s]

In [8]:
#直接使用我训练好的模型
model = Model.from_pretrained(repo_id)
test()

Some weights of the model checkpoint at distilgpt2 were not used when initializing GPT2Model: ['lm_head.weight']
- This IS expected if you are initializing GPT2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


0
tensor([1178,  257, 1200,   82, 1149,  611,  837, 2933])
tensor([691, 257, 262,  82, 262, 287, 220, 262])
10
tensor([15366, 27235,   355,  2644,   379,  7058,    12,   468])
tensor([ 518,   82,  355,  290,  262, 1267,   12,  705])
20
tensor([10735,   546,   837,  4035,  3923,  1645,  5924,   262])
tensor([ 388,  257, 1165, 4035, 2646,  326,  407,  262])
30
tensor([ 3972,   837,   286,   475, 16316,   829,   705,   705])
tensor([   83, 10997,   286,   475,   705,   293,   666,   705])
40
tensor([  357,  8531, 16631,  8886,   281, 26479,  8188,  2829])
tensor([2431, 8258,  837, 8886,  299,  286, 8188, 1621])
50
tensor([ 5688, 26597,   465,   290,  1275,   257,  3101, 10997])
tensor([  262, 26597,   290,   290,   318,   588,  3101, 10997])
0.28921568627450983
... the sum of the parts equals
 largely  the

eastwood is an icon of mov
iem iem

guided more by intellect than heart,
 his  and

this film is so slick, superficial
 and  and

a mixed bag of a comedy that
 ca  is

madonna still ca